<a href="https://colab.research.google.com/github/vittot/CLinkaRT-2023-Polimi/blob/main/AutomaticTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS

In [ ]:
!pip3 install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=b1d8e92afeeb46e9598371ad1ac654fb1596fddcda58a7a825bb323853d0cb03
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans


In [ ]:
from googletrans import Translator

In [ ]:
import re

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
tqdm.pandas()

# LOAD DATA

In [ ]:
import os, json
import pandas as pd
import numpy as np
import glob

path_to_json = '/content/drive/MyDrive/_Polimi/EVALITA/data_collection/'

language = 'Basque'

layers = ['layer1', 'layer2', 'layer3']

file_list = []
for l in tqdm(layers):
  json_pattern = os.path.join(path_to_json, language, l, '*.json')
  file_list = file_list + glob.glob(json_pattern)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
len(file_list)

10469

In [ ]:
dfs = [] # an empty list to store the data frames
for file in tqdm(file_list):
  authors_string = False
  authors_dict = False
  try:
    with open(file, 'r') as fp:
      d = json.load(fp)
    if d['authors'] == []:
      d['authors'] = ''
      i = [0]
    elif isinstance(d['authors'], str): #for some Spanish documents (es ES102568.json)
      authors_string = True
      i = [0]
    elif isinstance(d['authors'], dict): #for Basque
      authors_dict = True
      i = [0]
    else:
      i = list(range(len(d['authors'])))
    data = pd.DataFrame(d, index=i)
    if d['authors'] != '':
      try:
        data.authors = pd.DataFrame(data.authors.values.tolist())['author']
      except: #spanish has different format
        if authors_dict:
          data.authors = d['authors']['author']
        elif not authors_string:
          data.authors = pd.DataFrame(data.authors.values.tolist())[1]
      cols = list(data.columns) # columns are different in different languages
      cols.remove('authors')
      data = data.groupby(cols)['authors'].apply(','.join).reset_index()
    dfs.append(data) # append the data frame to the list
  except:
    print(file)
    break


res_df = pd.concat(dfs, ignore_index=True, axis=0) # concatenate all the data frames in the list.

  0%|          | 0/10469 [00:00<?, ?it/s]

In [ ]:
os.chdir('/content/drive/MyDrive/_Polimi/EVALITA')

# TRANSLATION

In [ ]:
translator = Translator()

In [ ]:
import time
#google translate API is limited to 15k characters
def translate_text(t):
  time.sleep(0.1)
  if len(t) < 15000:
    res = translator.translate(t, dest='it')
    return res.text
  else:
    r = ''
    while len(t) > 15000:
      time.sleep(0.1)
      try:
        substring = re.search("^(.|\t){1,14999}\s", t) # split at the last space before the 15k characters to avoid breaking words
        r = r + translator.translate(substring.group()).text
        t = t[substring.span()[1]:]
      except Exception as e:
        print(e)
        print(t)
        raise e
    r = r + translator.translate(t).text
    return r

In [ ]:
res_df['text_translated'] = ''

In [ ]:
res_df['text_translated'] = res_df['text'].progress_apply(lambda x: translate_text(x))

  0%|          | 0/1433 [00:00<?, ?it/s]

In [ ]:
res_df.to_csv('df_basque_translated.csv', index=False)

In [ ]:
res_df

,doi,publication_date,id,url,source,source_url,licence,language,type,description,text,authors,text_translated
0,,2019-03-01,EU100117,http://www.osagaiz.eus/article/view/248/298,Osagaiz,http://www.osagaiz.eus,CC-BY-NC,eu,journal,,3 hilabeteko haurra begirada galduagatik ospit...,"DeAlba,Igarzabal,Letona,Lacasta,Bereciartua,Un...",Il bambino di 3 mesi è stato ricoverato per pe...
1,,2014-MM-DD,EU100031,,ALTA-TXOSTENAK,https://www.euskadi.eus/,CC-BY-NC,eu,journal,,HEMATOLOGIA\nOSPITALERATZEKO ARRAZOIA.\nSukarr...,,EMATOLOGIA\nMOTIVO DEL RICOVERO.\nUna donna di...
2,,2008-10-03,EU100077,http://www.oeegunea.eus/biltzarrak/komunikazio...,Osasungoa Euskalduntzeko Erakundea,http://www.oeegunea.eus/,CC-BY-NC,eu,journal,,"50 urteko gizona, peritoneo-dialisia egiten du...","Odriozola,Gartzia,Intza",Un uomo di 50 anni in dialisi peritoneale si p...
3,,2014-MM-DD,EU100006,,ALTA-TXOSTENAK,https://www.euskadi.eus/,CC-BY-NC,eu,journal,,DERMATOLOGIA\nOSPITALERATZEKO/KONTSULTARAKO AR...,,DERMATOLOGIA\nMOTIVO DEL RICOVERO/CONSULENZA\n...
4,,2016-04-22,EU100108,http://www.oeegunea.eus/biltzarrak/komunikazio...,Osasungoa Euskalduntzeko Erakundea,http://www.oeegunea.eus/,CC-BY-NC,eu,journal,,"Gure pazientea emakume bat da 36 urterekin, tr...","Zabala,Carcaboso,Martinez,Barrutieta,Anparan,Saez",La nostra paziente è una donna di 36 anni che ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,,2019-01-17,EU100292,https://addi.ehu.es/handle/10810/30949,University of the Basque Country,https://addi.ehu.es/,CC-BY-NC,eu,other,Abstracts of Dissertations of Medicine,[EUS] Sarrera: Akiles tendoia edo tendoi kalka...,"Nájera Larrañaga, Nerea",[ITA] Introduzione: Sebbene il tendine di Achi...
1429,,2000-05-05,EU101106,http://www.oeegunea.eus/biltzarrak/komunikazio...,Osasungoa Euskalduntzeko Erakundea,http://www.oeegunea.eus,CC-BY-NC,eu,other,Annual conference of medicine,LARRITASUNERAKO JOERA ERIZAINTZAKO IKASLEENGAN...,"Egileak: X. ZUPIRIA , M.J. ALBERDI , M.J. URAN...",TENDENZE ALL'ANSIA NEGLI STUDENTI INFERMIERIST...
1430,,2011-06-06,EU100642,http://www.oeegunea.eus/biltzarrak/komunikazio...,Osasungoa Euskalduntzeko Erakundea,http://www.oeegunea.eus,CC-BY-NC,eu,other,Annual conference of medicine,4.0 Realâ8093Time PCRk kostu eraginkortasun er...,"Egileak: Jose Maria Manterola , I. Royo , J. A...",4.0 Realâ8093Time PCR ha un buon rapporto cost...
1431,,2011-06-06,EU100697,http://www.oeegunea.eus/biltzarrak/hitzaldiak/...,Osasungoa Euskalduntzeko Erakundea,http://www.oeegunea.eus,CC-BY-NC,eu,other,Annual conference of medicine,Minbiziaren bahetzea. Ebidentzia zientifikoan ...,Egilea: Nerea Muniozguren,Cancer screening. Based on scientific evidence...
